In [6]:
import scipy.stats as st
import numpy as np
import pandas as pd
from arcgis.geocoding import reverse_geocode
from arcgis.geometry import Geometry
from arcgis.gis import GIS


In [14]:
#import lstop data
lstops = pd.read_csv('../data/lstops.csv', delimiter=',', quotechar='"')
#check column
print(lstops['Location'].head())

lstops["Location"] = lstops["Location"].str.replace("(","") 
lstops["Location"] = lstops["Location"].str.replace(")","") 
lat_list = []
lon_list = []
for each in lstops["Location"]:
    x = each.split(",")
    lat = float(x[0])
    lon = float(x[1])
    lat_list.append(lat)
    lon_list.append(lon)

lstops["Lat"] = lat_list
lstops["Lon"] = lon_list

   STOP_ID DIRECTION_ID                        STOP_NAME  \
0    30162            W         18th (54th/Cermak-bound)   
1    30161            E                18th (Loop-bound)   
2    30022            N         35th/Archer (Loop-bound)   
3    30023            S       35th/Archer (Midway-bound)   
4    30214            S  35-Bronzeville-IIT (63rd-bound)   

           STATION_NAME           STATION_DESCRIPTIVE_NAME  MAP_ID   ADA  \
0                  18th                   18th (Pink Line)   40830  True   
1                  18th                   18th (Pink Line)   40830  True   
2           35th/Archer          35th/Archer (Orange Line)   40120  True   
3           35th/Archer          35th/Archer (Orange Line)   40120  True   
4  35th-Bronzeville-IIT  35th-Bronzeville-IIT (Green Line)   41120  True   

     RED   BLUE      G    BRN      P   Pexp      Y    Pnk      O  \
0  False  False  False  False  False  False  False   True  False   
1  False  False  False  False  False  False  F

In [8]:
#import ca to zip data
ca_to_zip = pd.read_csv('../data/ca_zip.csv',header=None)
ca_to_zip = ca_to_zip.drop(ca_to_zip.index[0])
ca_to_zip.columns = ["Community Area", "Zipcode", "Pop2010"]


In [9]:
#adapted from:
#https://gis.stackexchange.com/questions/352961/convert-lat-lon-to-zip-postal-code-using-python?
#fbclid=IwAR25An6-uDiY1Lpy5EVPUG15RcrRKj5fL1M86Bt3Svqy_iVRi1YLLfviRT0
df = lstops

gis = GIS()

def get_zip(df, lon_field, lat_field):
    location = reverse_geocode((Geometry({"x":float(df[lon_field]), "y":float(df[lat_field]), "spatialReference":{"wkid": 4326}})))
    return location['address']['Postal']

In [5]:
#apply zipcodes
lstops["zips"] = df.apply(get_zip, axis=1, lat_field='Lat', lon_field='Lon')
#merge based on ca_to_zip
lstops = lstops.merge(ca_to_zip, left_on="zips", right_on="Zipcode")
#output csv
lstops.to_csv("../data/lstops_localized.csv",index=False)